In [1]:
import requests
from bs4 import BeautifulSoup
#importing beautiful soap for scrapping the data

## Dependencies

`BBeautifulSoup` :    pip install BeautifulSoup

In [2]:

import pandas as pd
import numpy as np
import itertools #to create efficent looping to fetch more data in a go
import re 
import random 

### Creating BS4 Functions for scrapping

In [45]:
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1940-01-01,1980-12-31&num_votes=10000,&count=200" #1940 to 1980 200 listing

#https://www.imdb.com/search/title/?title_type=feature&release_date=2020-01-01,2021-12-31&num_votes=20000,&count=200 - 678 Files done .. processing time 45 min can be used for unlablled
#movie released b/w 2021 and 2020 having votes more than 20k @sanjay 

#https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2021-12-31&num_votes=60000,&count=200 @sanjay 2000 - 2021 6000votes filter 200 titles

#https://www.imdb.com/search/title/?title_type=feature&release_date=2005-01-01,2015-12-31&num_votes=30000,&count=200 @sanjay 3000 - 2005 -2015 3000votes filter 200 titles

#https://www.imdb.com/search/title/?title_type=feature&release_date=1980-01-01,2019-12-31&num_votes=500000,&count=200 @mona  500 Files done
#https://www.imdb.com/search/title/?title_type=tv_series&release_date=1980-01-01,2019-12-31&num_votes=80000,&count=200 @mona  500 Files done


#Sample files are put into Data_scrapped folder.
#Fetching only 600 movie listing as the processing time is great. once we are good with code we can change the above filter.
def getSoup(url):
    """
    Utility function this get soup function will fetch the above url which stored in url var.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def getReviews(soup):
    '''Function returns all reviews including postive and negative..'''
    
    # get a list of user ratings
    user_review_ratings = [tag.previous_element for tag in 
                           soup.find_all('span', attrs={'class': 'point-scale'})]        #can search div by inspect elementor
    
    
    # get the review tags
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    ans = []
    for i in range(5):
        ans.append(user_review_list[random.randint(0, len(user_review_list) -1)])
    links = ["https://www.imdb.com" + tag['href'] for tag in ans]
    return links


def getReviewText(review_url):
    '''Returns the user review text given the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find div tags with class text show-more__control
    tag = soup.find('div', attrs={'class': 'text show-more__control'})
    return tag.getText()

def getMovieTitle(review_url):
    '''Returns the movie title from the review url.'''
    # get the review_url's soup
    soup = getSoup(review_url)
    # find h1 tag
    tag = soup.find('h1')
    return list(tag.children)[1].getText()

def getNounChunks(user_review):
    # create the doc object
    doc = nlp(user_review)
    # get a list of noun_chunks
    noun_chunks = list(doc.noun_chunks)
    # convert noun_chunks from span objects to strings, otherwise it won't pickle
    noun_chunks_strlist = [chunk.text for chunk in noun_chunks]
    return noun_chunks_strlist
movies_soup = getSoup(url)

In [46]:
movie_tags = movies_soup.find_all('a', attrs={'class': None})

# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

print("There are a total of " + str(len(movie_tags)) + " movie titles")
print("Displaying 10 titles")
movie_tags[:10]

There are a total of 200 movie titles
Displaying 10 titles


['/title/tt0077651/',
 '/title/tt0080761/',
 '/title/tt0081505/',
 '/title/tt0073629/',
 '/title/tt0068646/',
 '/title/tt0074285/',
 '/title/tt0055928/',
 '/title/tt0078748/',
 '/title/tt0054215/',
 '/title/tt0072431/']

In [47]:
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
print("There are a total of " + str(len(movie_links)) + " movie user reviews")
print("Displaying 20 user reviews links")
movie_links[:20]

There are a total of 200 movie user reviews
Displaying 20 user reviews links


['https://www.imdb.com/title/tt0077651/reviews',
 'https://www.imdb.com/title/tt0080761/reviews',
 'https://www.imdb.com/title/tt0081505/reviews',
 'https://www.imdb.com/title/tt0073629/reviews',
 'https://www.imdb.com/title/tt0068646/reviews',
 'https://www.imdb.com/title/tt0074285/reviews',
 'https://www.imdb.com/title/tt0055928/reviews',
 'https://www.imdb.com/title/tt0078748/reviews',
 'https://www.imdb.com/title/tt0054215/reviews',
 'https://www.imdb.com/title/tt0072431/reviews',
 'https://www.imdb.com/title/tt0064757/reviews',
 'https://www.imdb.com/title/tt0070047/reviews',
 'https://www.imdb.com/title/tt0071807/reviews',
 'https://www.imdb.com/title/tt0072271/reviews',
 'https://www.imdb.com/title/tt0080453/reviews',
 'https://www.imdb.com/title/tt0073195/reviews',
 'https://www.imdb.com/title/tt0058150/reviews',
 'https://www.imdb.com/title/tt0066921/reviews',
 'https://www.imdb.com/title/tt0057076/reviews',
 'https://www.imdb.com/title/tt0076759/reviews']

In [48]:
movie_soups = [getSoup(link) for link in movie_links]

# get all 500 movie review links
movie_review_list = [getReviews(movie_soup) for movie_soup in movie_soups]

#movie_review_list = list(itertools.chain(*movie_review_list))
#print(len(movie_review_list))

#print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
#print("Displaying 10 reviews")
#movie_review_list[:10]

In [49]:

movie_review_list = list(itertools.chain(*movie_review_list))
print(len(movie_review_list))

print("There are a total of " + str(len(movie_review_list)) + " individual movie reviews")
print("Displaying 10 reviews")
movie_review_list[:10]

1000
There are a total of 1000 individual movie reviews
Displaying 10 reviews


['https://www.imdb.com/review/rw1550637/',
 'https://www.imdb.com/review/rw2150331/',
 'https://www.imdb.com/review/rw4423482/',
 'https://www.imdb.com/review/rw2496444/',
 'https://www.imdb.com/review/rw1550637/',
 'https://www.imdb.com/review/rw0176438/',
 'https://www.imdb.com/review/rw5279264/',
 'https://www.imdb.com/review/rw4025080/',
 'https://www.imdb.com/review/rw0176490/',
 'https://www.imdb.com/review/rw0176490/']

In [50]:
review_texts = [getReviewText(url) for url in movie_review_list]

# get movie name from the review link
movie_titles = [getMovieTitle(url) for url in movie_review_list]

# label each review with negative or positive

# construct a dataframe
df = pd.DataFrame({
             'user_review': review_texts })

In [51]:
df.head(5) #converting it into data frama

,user_review
0,Halloween starts in th small town of 'Haddonfi...
1,I must say that this isn't the type film of fi...
2,John Carpenter directed this low budget classi...
3,There was once a time where I wasn't crazy abo...
4,Halloween starts in th small town of 'Haddonfi...


In [52]:
text_list = [m for m in df['user_review']]
#text_list

In [53]:
text_list_length = [len(m.split()) for m in text_list]     
df['length'] = text_list_length
df

,user_review,length
0,Halloween starts in th small town of 'Haddonfi...,437
1,I must say that this isn't the type film of fi...,381
2,John Carpenter directed this low budget classi...,162
3,There was once a time where I wasn't crazy abo...,224
4,Halloween starts in th small town of 'Haddonfi...,437
...,...,...
995,The death of his brother brings Larry Talbot b...,621
996,"Modern horror fans will probably find this, th...",348
997,"A practical man (Lon Chaney, Jr.) returns to h...",130
998,"Man gets bitten by wolf, man kills the wolf, m...",150


In [54]:
df = df[df['length'] < 250]
df

,user_review,length
2,John Carpenter directed this low budget classi...,162
3,There was once a time where I wasn't crazy abo...,224
6,Up to this date the better of the young Friday...,95
8,"In a full moon night of a Friday, 13th, the in...",128
9,"In a full moon night of a Friday, 13th, the in...",128
...,...,...
991,"Yes, we in the 21st century always expect Lesl...",140
993,United Planet cruiser C57D is traveling to the...,184
994,"I liked how this started out, featuring some d...",210
997,"A practical man (Lon Chaney, Jr.) returns to h...",130


In [55]:
df.drop('length', axis=1, inplace=True)
df
#dropping the len row

C:\Users\sanja\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,user_review
2,John Carpenter directed this low budget classi...
3,There was once a time where I wasn't crazy abo...
6,Up to this date the better of the young Friday...
8,"In a full moon night of a Friday, 13th, the in..."
9,"In a full moon night of a Friday, 13th, the in..."
...,...
991,"Yes, we in the 21st century always expect Lesl..."
993,United Planet cruiser C57D is traveling to the...
994,"I liked how this started out, featuring some d..."
997,"A practical man (Lon Chaney, Jr.) returns to h..."


In [56]:
#converting only reviews to CSV & removing the index
df.to_csv('data_scrapped/data.csv', index=False) 

In [57]:
import csv

with open("data_scrapped/data.csv", "r") as f:
        reader = csv.reader(f)
        rownumber = 2639
        for row in reader:
             g=open(str(rownumber)+".txt","w")
             g.write(str(row))
             rownumber = rownumber + 1
             g.close()

#3076 is the last